In [1]:
import pandas as pd
df = pd.read_excel("C:/Users/artem/OneDrive/Desktop/WaitData.Published.xlsx", sheet_name='F3')
df.head()

,Wait,SumHowEarlyWaiting,AvgHowEarlyWaiting,LineCount0Strict,SumWaits,LineCount0,LineCount1,LineCount2,LineCount3,LineCount4,...,NumAddOnsToday,NumAddOnsLastW2,NumScheduledNextSlot,NumScheduledNextW2,SumTimeToCompleteNextSlot,SumTimeToCompleteNextW2,WithContrastCountWaiting,WithAndWithoutContrastCountWaiting,WithContrastCountInProgress,WithAndWithoutContrastCountInProgress
0,0,372,62,6,249,6,5,6,6,4,...,4,2,0,0,0,0,3,1,1,0
1,-3,335,67,3,195,5,4,6,6,5,...,4,1,2,0,32,0,2,1,2,0
2,-25,274,68,4,150,4,5,6,4,6,...,4,1,1,1,13,16,1,1,3,0
3,36,369,73,2,179,5,6,4,5,5,...,3,1,2,1,45,16,3,1,1,0
4,-21,438,73,5,192,6,5,5,6,5,...,3,0,2,1,32,15,4,0,0,1


In [2]:
X = df.drop(columns=['Wait'])  
Y = df['Wait']  
X.head()

,SumHowEarlyWaiting,AvgHowEarlyWaiting,LineCount0Strict,SumWaits,LineCount0,LineCount1,LineCount2,LineCount3,LineCount4,FlowCount2,...,NumAddOnsToday,NumAddOnsLastW2,NumScheduledNextSlot,NumScheduledNextW2,SumTimeToCompleteNextSlot,SumTimeToCompleteNextW2,WithContrastCountWaiting,WithAndWithoutContrastCountWaiting,WithContrastCountInProgress,WithAndWithoutContrastCountInProgress
0,372,62,6,249,6,5,6,6,4,2,...,4,2,0,0,0,0,3,1,1,0
1,335,67,3,195,5,4,6,6,5,3,...,4,1,2,0,32,0,2,1,2,0
2,274,68,4,150,4,5,6,4,6,4,...,4,1,1,1,13,16,1,1,3,0
3,369,73,2,179,5,6,4,5,5,3,...,3,1,2,1,45,16,3,1,1,0
4,438,73,5,192,6,5,5,6,5,2,...,3,0,2,1,32,15,4,0,0,1


In [3]:
from sklearn import linear_model
model = linear_model.LinearRegression()
model.fit(X, Y)
Ypred = model.predict(X) # use trained regression model to predict
r = Y-Ypred # compute prediction error (residual)
e = abs(r).mean() # compute model error
e


23.166288936347033

In [4]:
e = abs(r).median() # compute model error
e

19.126063446075705

In [5]:
from sklearn.feature_selection import RFE
from itertools import compress
# Run Python feature selection
if True: # just in case I want to disable this part
    print('\n>Python feature selection:')
    for nFeatures in range(1,4):
        rfe = RFE(model, n_features_to_select=nFeatures)
        X_rfe = rfe.fit_transform(X,Y) #transforming data using RFE   
        #Fitting the data to model
        model.fit(X_rfe,Y)
        #print(rfe.support_)
        #print(rfe.ranking_)
        cols = list(compress( X.columns, rfe.support_))
        model.fit(X[cols],Y)
        e = abs(Y-model.predict(X[cols])).mean()
        print(e, cols)



>Python feature selection:
32.50541172120655 ['CardiacCount']
31.224937706982736 ['CardiacCount', 'DelayedInLine']
31.25752894868932 ['CardiacCount', 'StartTime', 'DelayedInLine']


In [7]:
selected_features = []  
remaining_features = list(X.columns)  
best_error = float('inf') 

for _ in range(3):
    best_feature = None
    for feature in remaining_features:
        current_features = selected_features + [feature]
        model_clone = linear_model.LinearRegression()
        model_clone.fit(X[current_features], Y)
        error = abs(Y - model_clone.predict(X[current_features])).mean()
        if error < best_error:
            best_error = error
            best_feature = feature
    if best_feature is not None:
        selected_features.append(best_feature)
        remaining_features.remove(best_feature)
    
    print(f"Features: {selected_features}, Error: {best_error}")

Features: ['DelayedInLine'], Error: 31.227057699733514
Features: ['DelayedInLine', 'LineCount0Strict'], Error: 30.137965863467567
Features: ['DelayedInLine', 'LineCount0Strict', 'LineCount0'], Error: 28.69649063225338


In [10]:
selected_features = []  
remaining_features = list(X.columns)  
best_error = float('inf') 

for _ in range(15):
    best_feature = None
    for feature in remaining_features:
        current_features = selected_features + [feature]
        model_clone = linear_model.LinearRegression()
        model_clone.fit(X[current_features], Y)
        error = abs(Y - model_clone.predict(X[current_features])).mean()
        if error < best_error:
            best_error = error
            best_feature = feature
    if best_feature is not None:
        selected_features.append(best_feature)
        remaining_features.remove(best_feature)
    
    print(f"Features: {len(selected_features)}, Error: {best_error}")

Features: 1, Error: 31.227057699733514
Features: 2, Error: 30.137965863467567
Features: 3, Error: 28.69649063225338
Features: 4, Error: 28.2463095851323
Features: 5, Error: 28.147006842164515
Features: 6, Error: 24.989574734774777
Features: 7, Error: 24.659186407251333
Features: 8, Error: 24.329071006187288
Features: 9, Error: 24.071936786758197
Features: 10, Error: 23.84650949206529
Features: 11, Error: 23.69261939854713
Features: 12, Error: 23.627914296582233
Features: 13, Error: 23.579159670479896
Features: 14, Error: 23.54808179388156
Features: 15, Error: 23.520963512553575


In [8]:

models = []

for i in range(len(X.columns)):
    for j in range(i+1, len(X.columns)):
        for k in range(j+1, len(X.columns)):
            first = X.columns[i]
            second = X.columns[j]
            third = X.columns[k]
            model.fit(X[[first, second, third]], Y)
            y_pred = model.predict(X[[first, second, third]])
            error = abs(Y-model.predict(X[[first, second, third]])).mean()
            models.append((error, first, second, third))

models.sort(key=lambda x: x[0])

print(f"Best model: {models[0]}")
print("Top 5 models:")
for model in models[:5]:
    print(model)


Best model: (26.072745167091245, 'AheadCount', 'SumDelayWaitingByExamCode', 'NumCompletedToday')
Top 5 models:
(26.072745167091245, 'AheadCount', 'SumDelayWaitingByExamCode', 'NumCompletedToday')
(26.24225277074693, 'LineCount0', 'AheadCount', 'NumCompletedToday')
(26.252630216305025, 'AheadCount', 'NumCompletedToday', 'OutpatientWaitingCount')
(26.351994732330475, 'SumWaits', 'AheadCount', 'NumCompletedToday')
(26.36138559803822, 'AheadCount', 'SumWaitByTaskTypeLine', 'NumCompletedToday')
